# Capstone Project: Data Collection     

29/11/2020

We collect restaurant data via the Foursquare API.

In [1]:
import pandas as pd
import numpy as np
import requests 
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
#!pip install folium
import folium # map rendering library


In [2]:
# Coordinates of London
latitude = 51.5074
longitude = 0.1278

# create map of Toronto using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=10)
    
map_london

In [3]:
# @hidden_cell
CLIENT_ID = '34CRGBIL11CRLO5BNFSRKHWNRQT1GWTB4QKGYG5GLQYM3JKQ' # your Foursquare ID
CLIENT_SECRET = 'HRPY3HMJC1FSPYTC2VRVUANCY1JY20NBRA5400RQYOEOFCNB' # your Foursquare Secret
#VERSION = '20180605' # Foursquare API version
VERSION = '20201123' # Foursquare API version
RADIUS=500
LIMIT = 50 # A default Foursquare API limit value

In [4]:
# List of categories
afghan = '503288ae91d4c4b30a586d67'
african = '4bf58dd8d48988d1c8941735'
american = '4bf58dd8d48988d14e941735'
asian = '4bf58dd8d48988d142941735'
australian = '4bf58dd8d48988d169941735'
austrian = '52e81612bcbc57f1066b7a01'
bangladesh = '5e179ee74ae8e90006e9a746'
belgium = '52e81612bcbc57f1066b7a02'
cajun = '4bf58dd8d48988d17a941735'
caribbean = '4bf58dd8d48988d144941735'
caucasian = '5293a7d53cf9994f4e043a45'
czech = '52f2ae52bcbc57f1066b8b81'
dutch = '5744ccdfe4b0c0459246b4d0'
eastEurope = '4bf58dd8d48988d109941735'
english = '52e81612bcbc57f1066b7a05'
french = '4bf58dd8d48988d10c941735'
german = '4bf58dd8d48988d10d941735'
greek = '4bf58dd8d48988d10e941735'
hungary = '52e81612bcbc57f1066b79fa'
indian = '4bf58dd8d48988d10f941735'
italian = '4bf58dd8d48988d110941735'
jewish = '52e81612bcbc57f1066b79fd'
latin = '4bf58dd8d48988d1be941735'
meditarranean = '4bf58dd8d48988d1c0941735'
mexican = '4bf58dd8d48988d1c1941735'
middleEast = '4bf58dd8d48988d115941735'
modernEuropean = '52e81612bcbc57f1066b79f9'
pakistan = '52e81612bcbc57f1066b79f8'
polish = '52e81612bcbc57f1066b7a04'
portugal = '4def73e84765ae376e57713a'
russia = '5293a7563cf9994f4e043a44'
scandinavia = '4bf58dd8d48988d1c6941735'
scotland = '5744ccdde4b0c0459246b4a3'
slovak = '56aa371be4b08b9a8d57355a'
spain = '4bf58dd8d48988d150941735'
srilanka = '5413605de4b0ae91d18581a9'
swiss = '4bf58dd8d48988d158941735'
turkey = '4f04af1f2fb6e1c99f3db0bb'
ukraine = '52e928d0bcbc57f1066b7e96'

category_list = [afghan,african,american,asian,australian,austrian,bangladesh,belgium,cajun,caribbean,caucasian,czech,dutch,eastEurope,english,
                 french,german,greek,hungary,indian,italian,jewish,latin,meditarranean,mexican,middleEast,modernEuropean,pakistan,polish,
                 portugal,russia,scandinavia,scotland,slovak,spain,srilanka,swiss,turkey,ukraine] 

CATEGORIES = ','.join(category_list)

In [5]:
def getVenues(latitude, longitude, RADIUS=500):
    
    url = 'https://api.foursquare.com/v2/venues/search?&categoryId={}&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&intent=browse'.format(
            CATEGORIES,
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            latitude, 
            longitude, 
            RADIUS,
            LIMIT)
    
    response = requests.get(url)
    results = response.json()
    
    if response.status_code == 200:
        venues = results['response']['venues']
    else:
        venues = []
        print("HTTP-{}".format(response.status_code))
        
    venue_list = []
    
    number_of_venues = len(venues)
    
    for i in range(number_of_venues):
        
        try:
            venue_id = venues[i]['id']
            venue_name = venues[i]['name']
            lat = venues[i]['location']['lat']
            lng = venues[i]['location']['lng']
            category = venues[i]['categories'][0]['name']
            
            record = (latitude, longitude, number_of_venues, i, venue_id, venue_name, lat, lng, category)
        except:
            record = (latitude, longitude, number_of_venues, i, '', '', '', '', '')
        
        venue_list.append(record)
        
        
    return( venue_list )

# London

In [15]:
# Coordinates of London
latitude = 51.5074
longitude = -0.1278

# Box boundaries
y_max = latitude + 0.12
y_min = latitude - 0.12
x_max = longitude + 0.2
x_min = longitude - 0.2


lat_centre = (y_max+y_min)/2.0

y_step = 1.0/(110.574)
x_step = 1.0/(111.32*np.cos(2.0*np.pi*lat_centre/360.0))

num_x_points = int((x_max-x_min)/x_step)
num_y_points = int((y_max-y_min)/y_step)

lats = np.linspace(x_min,x_max,num_x_points)
longs = np.linspace(y_min,y_max,num_y_points)

' Create matrix of coordinate values '
coord_list = []
for x in longs:
    for y in lats:
        
        coord_list.append( (x,y) )

print("Number of points = {}".format(len(coord_list)))

Number of points = 702


In [16]:
# Coordinates of London
latitude = 51.5074
longitude = -0.1278

# create map of Toronto using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for coord in coord_list:
    lat = coord[0]
    lng = coord[1]
    label = '{} {}'.format(lat,lng)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=1,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

## Retrieve data from Foursquare

In [17]:
record_list = []

for coord in coord_list:
    
    lat = coord[0]
    lng = coord[1]
    
    venue_details = getVenues(lat, lng)
    
    record_list.extend(venue_details)
    
    print("lat = {} long = {} Number of Venues = {}".format(lat,lng,len(venue_details)))

    
col_names = ['lat_centre','long_centre','num_venues','venue_iter','venue_id','venue_name','lat_venue','long_venue','category']
df_venues = pd.DataFrame.from_records(record_list,columns=col_names)  

df_venues.to_csv('london_20201215_0925.csv',encoding='utf-8')

lat = 51.3874 long = -0.3278 Number of Venues = 2
lat = 51.3874 long = -0.312415384615 Number of Venues = 6
lat = 51.3874 long = -0.297030769231 Number of Venues = 14
lat = 51.3874 long = -0.281646153846 Number of Venues = 1
lat = 51.3874 long = -0.266261538462 Number of Venues = 2
lat = 51.3874 long = -0.250876923077 Number of Venues = 3
lat = 51.3874 long = -0.235492307692 Number of Venues = 0
lat = 51.3874 long = -0.220107692308 Number of Venues = 0
lat = 51.3874 long = -0.204723076923 Number of Venues = 4
lat = 51.3874 long = -0.189338461538 Number of Venues = 2
lat = 51.3874 long = -0.173953846154 Number of Venues = 3
lat = 51.3874 long = -0.158569230769 Number of Venues = 0
lat = 51.3874 long = -0.143184615385 Number of Venues = 2
lat = 51.3874 long = -0.1278 Number of Venues = 4
lat = 51.3874 long = -0.112415384615 Number of Venues = 3
lat = 51.3874 long = -0.0970307692308 Number of Venues = 6
lat = 51.3874 long = -0.0816461538462 Number of Venues = 0
lat = 51.3874 long = -0.066

lat = 51.4354 long = -0.204723076923 Number of Venues = 4
lat = 51.4354 long = -0.189338461538 Number of Venues = 7
lat = 51.4354 long = -0.173953846154 Number of Venues = 1
lat = 51.4354 long = -0.158569230769 Number of Venues = 32
lat = 51.4354 long = -0.143184615385 Number of Venues = 1
lat = 51.4354 long = -0.1278 Number of Venues = 26
lat = 51.4354 long = -0.112415384615 Number of Venues = 0
lat = 51.4354 long = -0.0970307692308 Number of Venues = 5
lat = 51.4354 long = -0.0816461538462 Number of Venues = 0
lat = 51.4354 long = -0.0662615384615 Number of Venues = 0
lat = 51.4354 long = -0.0508769230769 Number of Venues = 13
lat = 51.4354 long = -0.0354923076923 Number of Venues = 1
lat = 51.4354 long = -0.0201076923077 Number of Venues = 6
lat = 51.4354 long = -0.00472307692308 Number of Venues = 1
lat = 51.4354 long = 0.0106615384615 Number of Venues = 0
lat = 51.4354 long = 0.0260461538462 Number of Venues = 1
lat = 51.4354 long = 0.0414307692308 Number of Venues = 3
lat = 51.43

lat = 51.4834 long = -0.0816461538462 Number of Venues = 1
lat = 51.4834 long = -0.0662615384615 Number of Venues = 7
lat = 51.4834 long = -0.0508769230769 Number of Venues = 3
lat = 51.4834 long = -0.0354923076923 Number of Venues = 10
lat = 51.4834 long = -0.0201076923077 Number of Venues = 7
lat = 51.4834 long = -0.00472307692308 Number of Venues = 45
lat = 51.4834 long = 0.0106615384615 Number of Venues = 15
lat = 51.4834 long = 0.0260461538462 Number of Venues = 4
lat = 51.4834 long = 0.0414307692308 Number of Venues = 2
lat = 51.4834 long = 0.0568153846154 Number of Venues = 2
lat = 51.4834 long = 0.0722 Number of Venues = 6
lat = 51.493 long = -0.3278 Number of Venues = 4
lat = 51.493 long = -0.312415384615 Number of Venues = 1
lat = 51.493 long = -0.297030769231 Number of Venues = 2
lat = 51.493 long = -0.281646153846 Number of Venues = 7
lat = 51.493 long = -0.266261538462 Number of Venues = 15
lat = 51.493 long = -0.250876923077 Number of Venues = 50
lat = 51.493 long = -0.23

lat = 51.5314 long = 0.0260461538462 Number of Venues = 2
lat = 51.5314 long = 0.0414307692308 Number of Venues = 11
lat = 51.5314 long = 0.0568153846154 Number of Venues = 16
lat = 51.5314 long = 0.0722 Number of Venues = 2
lat = 51.541 long = -0.3278 Number of Venues = 0
lat = 51.541 long = -0.312415384615 Number of Venues = 3
lat = 51.541 long = -0.297030769231 Number of Venues = 11
lat = 51.541 long = -0.281646153846 Number of Venues = 2
lat = 51.541 long = -0.266261538462 Number of Venues = 1
lat = 51.541 long = -0.250876923077 Number of Venues = 11
lat = 51.541 long = -0.235492307692 Number of Venues = 0
lat = 51.541 long = -0.220107692308 Number of Venues = 2
lat = 51.541 long = -0.204723076923 Number of Venues = 42
lat = 51.541 long = -0.189338461538 Number of Venues = 17
lat = 51.541 long = -0.173953846154 Number of Venues = 24
lat = 51.541 long = -0.158569230769 Number of Venues = 12
lat = 51.541 long = -0.143184615385 Number of Venues = 50
lat = 51.541 long = -0.1278 Number 

lat = 51.589 long = -0.266261538462 Number of Venues = 0
lat = 51.589 long = -0.250876923077 Number of Venues = 9
lat = 51.589 long = -0.235492307692 Number of Venues = 6
lat = 51.589 long = -0.220107692308 Number of Venues = 11
lat = 51.589 long = -0.204723076923 Number of Venues = 4
lat = 51.589 long = -0.189338461538 Number of Venues = 2
lat = 51.589 long = -0.173953846154 Number of Venues = 3
lat = 51.589 long = -0.158569230769 Number of Venues = 11
lat = 51.589 long = -0.143184615385 Number of Venues = 22
lat = 51.589 long = -0.1278 Number of Venues = 8
lat = 51.589 long = -0.112415384615 Number of Venues = 23
lat = 51.589 long = -0.0970307692308 Number of Venues = 16
lat = 51.589 long = -0.0816461538462 Number of Venues = 9
lat = 51.589 long = -0.0662615384615 Number of Venues = 12
lat = 51.589 long = -0.0508769230769 Number of Venues = 0
lat = 51.589 long = -0.0354923076923 Number of Venues = 7
lat = 51.589 long = -0.0201076923077 Number of Venues = 22
lat = 51.589 long = -0.004

In [18]:
print("Limit breached in {} sites".format(sum(df_venues['num_venues']==50)/50))
np.unique(df_venues[df_venues['num_venues']==50][['lat_centre','long_centre']], axis=0)

Limit breached in 34 sites


array([[ 5.14642000e+01, -1.12415385e-01],
       [ 5.14930000e+01, -2.50876923e-01],
       [ 5.14930000e+01, -2.35492308e-01],
       [ 5.14930000e+01, -1.89338462e-01],
       [ 5.14930000e+01, -1.73953846e-01],
       [ 5.14930000e+01, -1.43184615e-01],
       [ 5.14930000e+01, -9.70307692e-02],
       [ 5.15026000e+01, -2.20107692e-01],
       [ 5.15026000e+01, -1.89338462e-01],
       [ 5.15026000e+01, -1.58569231e-01],
       [ 5.15026000e+01, -1.12415385e-01],
       [ 5.15026000e+01, -9.70307692e-02],
       [ 5.15026000e+01, -8.16461538e-02],
       [ 5.15026000e+01, -2.01076923e-02],
       [ 5.15122000e+01, -1.89338462e-01],
       [ 5.15122000e+01, -1.58569231e-01],
       [ 5.15122000e+01, -1.43184615e-01],
       [ 5.15122000e+01, -1.27800000e-01],
       [ 5.15122000e+01, -1.12415385e-01],
       [ 5.15122000e+01, -9.70307692e-02],
       [ 5.15122000e+01, -8.16461538e-02],
       [ 5.15218000e+01, -2.04723077e-01],
       [ 5.15218000e+01, -1.58569231e-01],
       [ 5.

## Calculate new site locations at breached points

In [19]:
zoom_coord_list = np.unique(df_venues[df_venues['num_venues']==50][['lat_centre','long_centre']], axis=0).tolist()

In [20]:
coord_list = []

for coord in zoom_coord_list:
    
    x = coord[1]
    y = coord[0]
    
    y_step = 1.0/(110.574)
    x_step = 1.0/(111.32*np.cos(2.0*np.pi*y/360.0))

    xleft = x - (x_step/4.0)
    xright = x + (x_step/4.0)
    ytop = y + (y_step/4.0)
    ybottom = y - (y_step/4.0)
    
    coord_list.append( (y,x) )
    coord_list.append( (ytop,xleft) )
    coord_list.append( (ybottom,xleft) )
    coord_list.append( (ytop,xright) )
    coord_list.append( (ybottom,xright) )

In [21]:
# Coordinates of London
latitude = 51.5074
longitude = -0.1278

# create map of Toronto using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for coord in coord_list:
    lat = coord[0]
    lng = coord[1]
    label = '{} {}'.format(lat,lng)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=1,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

## Retrieve info from Foursquare

In [22]:
record_list = []

for coord in coord_list:
    
    lat = coord[0]
    lng = coord[1]
    
    venue_details = getVenues(lat, lng, RADIUS=250)
    
    record_list.extend(venue_details)
    
    print("lat = {} long = {} Number of Venues = {}".format(lat,lng,len(venue_details)))

    
col_names = ['lat_centre','long_centre','num_venues','venue_iter','venue_id','venue_name','lat_venue','long_venue','category']
df_venues = pd.DataFrame.from_records(record_list,columns=col_names)  

df_venues.to_csv('london_20201211_0935.csv',encoding='utf-8')

lat = 51.4642 long = -0.112415384615 Number of Venues = 50
lat = 51.4664609293 long = -0.116020144299 Number of Venues = 9
lat = 51.4619390707 long = -0.116020144299 Number of Venues = 49
lat = 51.4664609293 long = -0.108810624932 Number of Venues = 0
lat = 51.4619390707 long = -0.108810624932 Number of Venues = 37
lat = 51.493 long = -0.250876923077 Number of Venues = 20
lat = 51.4952609293 long = -0.254483959667 Number of Venues = 9
lat = 51.4907390707 long = -0.254483959667 Number of Venues = 21
lat = 51.4952609293 long = -0.247269886487 Number of Venues = 9
lat = 51.4907390707 long = -0.247269886487 Number of Venues = 0
lat = 51.493 long = -0.235492307692 Number of Venues = 38
lat = 51.4952609293 long = -0.239099344283 Number of Venues = 19
lat = 51.4907390707 long = -0.239099344283 Number of Venues = 3
lat = 51.4952609293 long = -0.231885271102 Number of Venues = 4
lat = 51.4907390707 long = -0.231885271102 Number of Venues = 17
lat = 51.493 long = -0.189338461538 Number of Venues

lat = 51.5195390707 long = -0.0780368365586 Number of Venues = 50
lat = 51.5218 long = -0.0662615384615 Number of Venues = 4
lat = 51.5240609293 long = -0.0698708557491 Number of Venues = 26
lat = 51.5195390707 long = -0.0698708557491 Number of Venues = 50
lat = 51.5240609293 long = -0.062652221174 Number of Venues = 2
lat = 51.5195390707 long = -0.062652221174 Number of Venues = 16
lat = 51.5314 long = -0.1278 Number of Venues = 13
lat = 51.5336609293 long = -0.131410078364 Number of Venues = 2
lat = 51.5291390707 long = -0.131410078364 Number of Venues = 19
lat = 51.5336609293 long = -0.124189921636 Number of Venues = 28
lat = 51.5291390707 long = -0.124189921636 Number of Venues = 20
lat = 51.5314 long = -0.112415384615 Number of Venues = 7
lat = 51.5336609293 long = -0.116025462979 Number of Venues = 3
lat = 51.5291390707 long = -0.116025462979 Number of Venues = 13
lat = 51.5336609293 long = -0.108805306252 Number of Venues = 39
lat = 51.5291390707 long = -0.108805306252 Number of

In [23]:
print("Limit breached in {} sites".format(sum(df_venues['num_venues']==50)/50))
np.unique(df_venues[df_venues['num_venues']==50][['lat_centre','long_centre']], axis=0)

Limit breached in 25 sites


array([[ 5.14642000e+01, -1.12415385e-01],
       [ 5.14930000e+01, -1.73953846e-01],
       [ 5.15003391e+01, -1.62177027e-01],
       [ 5.15048609e+01, -9.34229728e-02],
       [ 5.15099391e+01, -1.39576059e-01],
       [ 5.15099391e+01, -1.31408557e-01],
       [ 5.15099391e+01, -1.24191443e-01],
       [ 5.15122000e+01, -1.89338462e-01],
       [ 5.15122000e+01, -1.27800000e-01],
       [ 5.15122000e+01, -8.16461538e-02],
       [ 5.15144609e+01, -1.62177787e-01],
       [ 5.15144609e+01, -1.39576059e-01],
       [ 5.15144609e+01, -1.31408557e-01],
       [ 5.15144609e+01, -1.24191443e-01],
       [ 5.15144609e+01, -9.34222126e-02],
       [ 5.15144609e+01, -7.80375972e-02],
       [ 5.15195391e+01, -1.08806067e-01],
       [ 5.15195391e+01, -7.80368366e-02],
       [ 5.15195391e+01, -6.98708557e-02],
       [ 5.15218000e+01, -1.12415385e-01],
       [ 5.15240609e+01, -1.08806067e-01],
       [ 5.15240609e+01, -7.80368366e-02],
       [ 5.15410000e+01, -1.43184615e-01],
       [ 5.

## Calculate new site locations at breached points

In [24]:
zoom_coord_list = np.unique(df_venues[df_venues['num_venues']==50][['lat_centre','long_centre']], axis=0).tolist()

In [25]:
coord_list = []

for coord in zoom_coord_list:
    
    x = coord[1]
    y = coord[0]
    
    y_step = 1.0/(110.574)
    x_step = 1.0/(111.32*np.cos(2.0*np.pi*y/360.0))

    xleft = x - (x_step/8.0)
    xright = x + (x_step/8.0)
    ytop = y + (y_step/8.0)
    ybottom = y - (y_step/8.0)
    
    coord_list.append( (y,x) )
    coord_list.append( (ytop,xleft) )
    coord_list.append( (ybottom,xleft) )
    coord_list.append( (ytop,xright) )
    coord_list.append( (ybottom,xright) )

In [26]:
# Coordinates of London
latitude = 51.5074
longitude = -0.1278

# create map of Toronto using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for coord in coord_list:
    lat = coord[0]
    lng = coord[1]
    label = '{} {}'.format(lat,lng)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=1,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

## Retrieve info from Foursquare

In [27]:
record_list = []

for coord in coord_list:
    
    lat = coord[0]
    lng = coord[1]
    
    venue_details = getVenues(lat, lng, RADIUS=100)
    
    record_list.extend(venue_details)
    
    print("lat = {} long = {} Number of Venues = {}".format(lat,lng,len(venue_details)))

    
col_names = ['lat_centre','long_centre','num_venues','venue_iter','venue_id','venue_name','lat_venue','long_venue','category']
df_venues = pd.DataFrame.from_records(record_list,columns=col_names)  

df_venues.to_csv('london_20201215_1000.csv',encoding='utf-8')


lat = 51.4642 long = -0.112415384615 Number of Venues = 14
lat = 51.4653304647 long = -0.114217764457 Number of Venues = 4
lat = 51.4630695353 long = -0.114217764457 Number of Venues = 14
lat = 51.4653304647 long = -0.110613004773 Number of Venues = 0
lat = 51.4630695353 long = -0.110613004773 Number of Venues = 0
lat = 51.493 long = -0.173953846154 Number of Venues = 3
lat = 51.4941304647 long = -0.175757364449 Number of Venues = 25
lat = 51.4918695353 long = -0.175757364449 Number of Venues = 3
lat = 51.4941304647 long = -0.172150327859 Number of Venues = 0
lat = 51.4918695353 long = -0.172150327859 Number of Venues = 5
lat = 51.5003390707 long = -0.16217702717 Number of Venues = 11
lat = 51.5014695353 long = -0.163980835879 Number of Venues = 4
lat = 51.499208606 long = -0.163980835879 Number of Venues = 14
lat = 51.5014695353 long = -0.160373218462 Number of Venues = 5
lat = 51.499208606 long = -0.160373218462 Number of Venues = 5
lat = 51.5048609293 long = -0.0934229728296 Number 

HTTP-429
lat = 51.5421304647 long = -0.0101394264191 Number of Venues = 0
HTTP-429
lat = 51.544391394 long = -0.00652840715196 Number of Venues = 0
HTTP-429
lat = 51.5421304647 long = -0.00652840715196 Number of Venues = 0


## Combine all files into single file

In [28]:
import os
files = os.listdir('.')

file_pattern = 'london_'

venue_files = []

' Obtain all files holding venue info '
for filename in files:
    if file_pattern in filename:
        venue_files.append(filename)

' Read in files into dataframe '
df_venues = pd.DataFrame()
for filename in venue_files:
    df = pd.read_csv(filename,encoding='utf-8')
    print( "File: {} Records: {}".format(filename,len(df)) )
    df_venues = df_venues.append(df, ignore_index = True) 

File: london_20201215_0925.csv Records: 6997
File: london_20201215_1000.csv Records: 1001
File: london_20201211_0935.csv Records: 3969


## Remove duplicate entries and clean up data

In [29]:
df_venues.sort_values("venue_id", inplace = True)
df_venues.drop_duplicates(subset ="venue_id", keep = False, inplace = True)
df_venues.to_csv('london_all.csv',encoding='utf-8')

In [30]:
' Keep only venues with "Restaurant" in their category title '
df_venues['restaurant'] = df_venues['category'].apply(lambda x: True if 'Restaurant' in x else False)
df_venues = df_venues[df_venues['restaurant'] == True]
df_venues.drop('restaurant', axis=1, inplace=True)
df_venues.to_csv('london_restaurants.csv',encoding='utf-8')

In [31]:
from folium.plugins import FastMarkerCluster

# Coordinates of London
latitude = 51.5074
longitude = -0.1278

# create map of Zurich using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=10)

FastMarkerCluster(data=list(zip(df_venues['lat_venue'],df_venues['long_venue']))).add_to(map_london)

folium.LayerControl().add_to(map_london);

map_london
